In [ ]:
## import cv2
import mss
import time
import numpy as np
import import_ipynb
import pyautogui as pag
import matplotlib.pyplot as plt
import re

from PIL import ImageFont, ImageDraw, Image
from VideoKeyboard import *
from Ngram import*
from datetime import datetime
from collections import deque
from Gaze import Detector, Predictor
from Models import FullModel
from utils import get_config, clamp_value

# Read config.ini file
SETTINGS, COLOURS, EYETRACKER, TF = get_config("config.ini")

# Start
OutputFeedback = 0
keyboard = Controller()
listaRec = []
listaRecAnt = []
stspace = False
alpha = 0.4
x_hat = 0
y_hat = 0
RecomendationBox = None
RecomendationBoxAnt = None
finalTextAnt = None
buttonAnt = None
font_input = ImageFont.truetype("C:\Windows\Fonts\\arialbd.ttf", 35)
font_input_small = ImageFont.truetype("C:\Windows\Fonts\\arialbd.ttf", 30)
font_output = ImageFont.truetype("C:\Windows\Fonts\\arialbd.ttf", 45)

with mss.mss() as sct:
    mon = sct.monitors[EYETRACKER["monitor_num"]]
    w, h = mon["width"], mon["height"]
    monitor = {
        "top": mon["top"],
        "left": mon["left"],
        "width": w,
        "height": h,
        "mon": EYETRACKER["monitor_num"],
    }

    # Load trained model
    detector = Detector(output_size=SETTINGS["image_size"])
    predictor = Predictor(
        FullModel,
        model_data="trained_models/eyetracking_model.pt",
        config_file="trained_models/eyetracking_config.json",
    )
    screen_errors = region_map = np.load("trained_models/eyetracking_errors.npy")

    track_x = deque(
        [0] * SETTINGS["avg_window_length"], maxlen=SETTINGS["avg_window_length"]
    )
    track_y = deque(
        [0] * SETTINGS["avg_window_length"], maxlen=SETTINGS["avg_window_length"]
    )
    track_error = deque(
        [0] * (SETTINGS["avg_window_length"] * 2),
        maxlen=SETTINGS["avg_window_length"] * 2,
    )

    videoWriter = None
    if EYETRACKER["write_to_disk"]:
        # FIXME: fix video codec on windows
        date_time = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
        fourcc = cv2.VideoWriter_fourcc(*"avc1")
        videoWriter = cv2.VideoWriter(
            "media/recordings/{}.mp4".format(date_time),
            fourcc,
            EYETRACKER["tracker_frame_rate"],
            (
                int(w * EYETRACKER["screen_capture_scale"]),
                int(h * EYETRACKER["screen_capture_scale"]),
            ),
        )

    last_time = time.time()
    while True:
        if cv2.waitKey(1) & 0xFF == 27:  # wait for escape key
            detector.close()
            cv2.destroyAllWindows()
            break
            
        cur_time = time.time()

        if (cur_time - last_time) >= 1 / EYETRACKER["tracker_frame_rate"]:
            fps = 1 / (cur_time - last_time)
            last_time = cur_time

            # Get camera data
            frame, l_eye, r_eye, face, face_align, head_pos, angle = detector.get_frame()
            
            # Get screenshot
            if finalText != finalTextAnt:
                listaRec = recPalavra(finalText)
                finalTextAnt = finalText
            if len (listaRec) == 0:
                    listaRec = listaRecAnt
            if len(listaRec) < 10:
                for i in range (10 - len(listaRec)):
                    listaRec.append("")
            listaRecAnt = listaRec
            
            # Show Keyboard
            img = drawAll(frame, buttonList, alpha)

            # Show Recomendation Box
            if (EYETRACKER["show_recomendation"] == True):
                img = drawRec(img,  alpha, listaRec) 
            
            # Display and focus Space Box (down)
            cv2.rectangle(img, (40, 710), (1230, 750), (COLOURS["purple"]), cv2.FILLED)
            cv2.circle(img, (640, 755), 60, (COLOURS["purple"]), cv2.FILLED)
            cv2.putText(img, "\/", (630, 712),
                        cv2.FONT_HERSHEY_PLAIN, 0.9, (COLOURS["white"]), 4)
            if 40 < x_hat < 1230 and 710 < y_hat:
                cv2.rectangle(img, (40, 690), (1230, 750), (COLOURS["purple"]), cv2.FILLED)
                pil_image = Image.fromarray(img)
                draw = ImageDraw.Draw(pil_image)
                draw.text((590, 685), ("espaço"), font=font_input_small)
                img = np.asarray(pil_image)
                timebox_e += 1
                if timebox_e >= EYETRACKER["timebox_espaco"]:
                    cv2.rectangle(img, (40, 690), (1230, 750), (COLOURS["darkpurple"]), cv2.FILLED)
                    finalText +=  ' '
                    timebox_e = 0
            else:
                timebox_e = 0


            # Display and focus Del Box (right)
            cv2.rectangle(img, (1270, 160), (1280, 670), (COLOURS["purple"]), cv2.FILLED)
            cv2.circle(img, (1315, 410), 60, (COLOURS["purple"]), cv2.FILLED)
            cv2.putText(img, ">", (1262, 422),
                        cv2.FONT_HERSHEY_PLAIN, 1.8, (COLOURS["white"]), 4)
            if 1270 < x_hat and 160 < y_hat < 670:
                cv2.rectangle(img, (1250, 160), (1280, 670), (COLOURS["purple"]), cv2.FILLED)
                pil_image = Image.fromarray(img)
                draw = ImageDraw.Draw(pil_image)
                draw.text((1254, 330), ("a"), font=font_input_small)
                draw.text((1254, 360), ("p"), font=font_input_small)
                draw.text((1254, 390), ("a"), font=font_input_small)
                draw.text((1254, 420), ("g"), font=font_input_small)
                draw.text((1254, 450), ("a"), font=font_input_small)
                draw.text((1258, 480), ("r"), font=font_input_small)
                img = np.asarray(pil_image)
                timebox_a += 1
                if timebox_a >= EYETRACKER["timebox_apagar"]:
                    cv2.rectangle(img, (1250, 160), (1280, 670), (COLOURS["darkpurple"]), cv2.FILLED)
                    finalText = finalText[:-1]
                    timebox_a = 0
            else:
                timebox_a = 0

            # Display and focus Del Box (left)
            cv2.rectangle(img, (0, 160), (10, 670), (COLOURS["purple"]), cv2.FILLED)
            cv2.circle(img, (-35, 410), 60, (COLOURS["purple"]), cv2.FILLED)
            cv2.putText(img, "<", (3, 422),
                        cv2.FONT_HERSHEY_PLAIN, 1.8, (COLOURS["white"]), 4)
            if x_hat < 30 and 160 < y_hat < 670:
                cv2.rectangle(img, (0, 160), (30, 670), (COLOURS["purple"]), cv2.FILLED)
                pil_image = Image.fromarray(img)
                draw = ImageDraw.Draw(pil_image)
                draw.text((9, 280), ("r"), font=font_input_small)
                draw.text((5, 310), ("e"), font=font_input_small)
                draw.text((9, 340), ("i"), font=font_input_small)
                draw.text((5, 370), ("n"), font=font_input_small)
                draw.text((9, 400), ("i"), font=font_input_small)
                draw.text((5, 430), ("c"), font=font_input_small)
                draw.text((9, 460), ("i"), font=font_input_small)
                draw.text((5, 490), ("a"), font=font_input_small)
                draw.text((9, 520), ("r"), font=font_input_small)
                img = np.asarray(pil_image)
                timebox_re += 1
                if timebox_re >= EYETRACKER["timebox_reiniciar"]:
                    cv2.rectangle(img, (1250, 160), (1280, 670), (COLOURS["darkpurple"]), cv2.FILLED)
                    finalText = ''
                    timebox_re = 0
            else:
                timebox_re = 0
                
            # Display the Output box 
            cv2.rectangle(img, (150, 40), (1230, 140), (COLOURS["purple"]), cv2.FILLED)
            img = cv2.addWeighted(frame, alpha, img, 1 - alpha, 0)
            pil_image = Image.fromarray(img)
            draw = ImageDraw.Draw(pil_image)
            OutputFeedback += 1
            if len(finalText) <= 41:
                stspace = False
                if OutputFeedback % 2 == 0:
                    draw.text((160, 43), (finalText + "|"), font=font_output)
                else:
                    draw.text((160, 43), (finalText), font=font_output)
            else:
                if OutputFeedback % 2 == 0:
                    if finalText[-1:] == " ":
                        draw.text((160, 43), (finalText[:41]), font=font_output)
                        draw.text((160, 85), (finalText[41:] + "|"), font=font_output)
                    else:
                        for i in range (len(finalText)):
                            if finalText[len(finalText)-i-1:len(finalText)-i] == " " and stspace == False:
                                stspace = True
                                lineControl = len(finalText)-i
                        draw.text((160, 43), (finalText[:lineControl]), font=font_output)
                        draw.text((160, 85), (finalText[lineControl:] + "|"), font=font_output)
                else:
                    if finalText[-1:] == " ":
                        draw.text((160, 43), (finalText[:41]), font=font_output)
                        draw.text((160, 85), (finalText[41:]), font=font_output)
                    else:
                        for i in range (len(finalText)):
                            if finalText[len(finalText)-i-1:len(finalText)-i] == " " and stspace == False:
                                stspace = True
                                lineControl = len(finalText)-i 
                        draw.text((160, 43), (finalText[:lineControl]), font=font_output)
                        draw.text((160, 85), (finalText[lineControl:]), font=font_output)
            img = np.asarray(pil_image)
            
            # Keyboard focus whitch browser and write
            for button in buttonList:
                button.text = button.text.upper()
                button_x, button_y = button.pos
                button_w, button_h = button.size      
                if button_x < x_hat < button_x + button_w and button_y < y_hat < button_y + button_h:
                    cv2.rectangle(img, (button_x - 5, button_y - 5), (button_x + button_w + 5, button_y + button_h + 5), (COLOURS["purple"]), cv2.FILLED)
                    #pil_image = Image.fromarray(img)
                    #draw = ImageDraw.Draw(pil_image)
                    #img = np.asarray(pil_image)
                    #draw.text((button_x + 20, button_y + 10), button.text, font=font_output)
                    cv2.putText(img, button.text, (button_x + 30, button_y + 75),
                                cv2.FONT_HERSHEY_PLAIN, 4, (COLOURS["white"]), 4)
                    #img = np.asarray(pil_image)
                    
                    #Timer
                    if button.text == buttonAnt:
                        timebox_k += 1
                    else:
                        buttonAnt = button.text
                        timebox_k = 0
                        
                    if timebox_k >= EYETRACKER["timebox_keyboard"]:
                        if button.text != "<" and button.text != "_" and button.text != "-":
                            if finalText != "":
                                finalText += button.text.lower()
                            else:
                                finalText = button.text
                            timebox_k = 0
                        else:
                            # delete                 
                            if button.text == "<":
                                finalText = finalText[:-1]
                            # insert enter               
                            if button.text == "_":
                                finalText +=  ' ' 
                            if button.text == "-":
                                finalText = '' 
                            timebox_k = 0
        
            
            # Recomendation focus whitch brower and write
            if (EYETRACKER["show_recomendation"] == True):
                xmin_1, xmax_1, ymin_1, ymax_1 = 45, 270, 160, 225
                xmin_2, xmax_2, ymin_2, ymax_2 = 45, 270, 240, 305
                for j in range (len(listaRec)):
                    if j < (len(listaRec)/2):
                        if xmin_1 < x_hat < xmax_1 and ymin_1 < y_hat < ymax_1:
                            RecomendationBox = j
                            if listaRec[j] != "":
                                cv2.rectangle(img, (xmin_1, ymin_1), (xmax_1, ymax_1), (COLOURS["purple"]), cv2.FILLED)
                                pil_image = Image.fromarray(img)
                                draw = ImageDraw.Draw(pil_image)
                                if len(listaRec[j]) > 11:
                                    listaRecAbrev = listaRec[j][0:8] + "..."
                                    draw.text((int(xmax_1-110-(((len(listaRecAbrev))*18)/2)), 170), listaRecAbrev, font=font_input)
                                else:
                                    draw.text((int(xmax_1-110-(((len(listaRec[j]))*18)/2)), 170), listaRec[j], font=font_input)
                            img = np.asarray(pil_image)
                        xmin_1 += 240
                        xmax_1 += 240
                    else:
                        if xmin_2 < x_hat < xmax_2 and ymin_2 < y_hat < ymax_2:
                            RecomendationBox = j
                            if listaRec[j] != "":
                                cv2.rectangle(img, (xmin_2, ymin_2), (xmax_2, ymax_2), (COLOURS["purple"]), cv2.FILLED)
                                pil_image = Image.fromarray(img)
                                draw = ImageDraw.Draw(pil_image)
                                if len(listaRec[j]) > 11:
                                    listaRecAbrev = listaRec[j][0:8] + "..."
                                    draw.text((int(xmax_2-110-(((len(listaRecAbrev))*18)/2)), 250), listaRecAbrev, font=font_input)
                                else:
                                    draw.text((int(xmax_2-110-(((len(listaRec[j]))*18)/2)), 250), listaRec[j], font=font_input)
                            img = np.asarray(pil_image)
                        xmin_2 += 240
                        xmax_2 += 240
    
                if 45 < x_hat < 1270 and 160 < y_hat < 305:
                    if RecomendationBox == RecomendationBoxAnt and RecomendationBox != None:
                        timebox_r += 1
                    if RecomendationBox != RecomendationBoxAnt:
                        RecomendationBoxAnt = RecomendationBox
                        timebox_r = 0
                    if timebox_r >= EYETRACKER["timebox_recomendation"] and RecomendationBox != None:
                        if listaRec[RecomendationBox] != "":
                            if finalText[-1:] == " ":
                                finalText += listaRec[RecomendationBox] + " "
                            elif finalText != "":
                                finalText += " " + listaRec[RecomendationBox]
                            else: 
                                finalText = listaRec[RecomendationBox].title() + " "
                            timebox_r = 0
                            RecomendationBox = None
            
            screenshot = np.array(img)
            overlay = screenshot.copy()

            # Overlays
            x_hat, y_hat = predictor.predict(
                face, l_eye, r_eye, head_pos, head_angle=angle
            )

            track_x.append(x_hat)
            track_y.append(y_hat)

            x_hat_clamp = clamp_value(x_hat, w)
            y_hat_clamp = clamp_value(y_hat, h)
            error = screen_errors[int(x_hat_clamp) - 1][int(y_hat_clamp) - 1]
            track_error.append(error * 0.75)

            weights = np.arange(1, SETTINGS["avg_window_length"] + 1)
            weights_error = np.arange(1, (SETTINGS["avg_window_length"] * 2) + 1)

            cv2.circle(
                overlay,
                (
                    int(np.average(track_x, weights=weights)),
                    int(np.average(track_y, weights=weights)),
                ),
                int(np.average(track_error, weights=weights_error)),
                (255, 255, 255, 50),
                -1,
            )

            if finalText == finalTextAnt:
                cv2.circle(
                    screenshot,
                    (
                        int(np.average(track_x, weights=weights)),
                        int(np.average(track_y, weights=weights)),
                    ),
                    int(np.average(track_error, weights=weights_error)),
                    COLOURS["green"],
                    5,
                )
            else:
                cv2.circle(
                    screenshot,
                    (
                        int(np.average(track_x, weights=weights)),
                        int(np.average(track_y, weights=weights)),
                    ),
                    int(np.average(track_error, weights=weights_error)),
                    COLOURS["red"],
                    5,
                )
                
            screenshot = cv2.addWeighted(overlay, 0.3, screenshot, 0.7, 0)

            cv2.putText(
                screenshot,
                "fps: {}".format(round(fps, 1)),
                (10, 150),
                cv2.FONT_HERSHEY_PLAIN,
                1.5,
                COLOURS["green"],
            )

            if EYETRACKER["show_webcam"]:
                large_size = SETTINGS["image_size"]

                screenshot[0:large_size, 0:large_size, 0:3] = cv2.resize(
                    face_align, (large_size, large_size)
                )

                screenshot[
                    0:large_size,
                    large_size : large_size * 2,
                    0:3,
                ] = cv2.resize(
                    np.repeat(head_pos[:, :, np.newaxis], 3, axis=2),
                    (large_size, large_size),
                )

                screenshot[
                    large_size : large_size * 2,
                    0:large_size,
                    0:3,
                ] = cv2.resize(l_eye, (large_size, large_size))

                screenshot[
                    large_size : large_size * 2,
                    large_size : large_size * 2,
                    0:3,
                ] = cv2.resize(r_eye, (large_size, large_size))

            # Resize and write frame
            screenshot = cv2.resize(
                screenshot,
                (
                    int(w * EYETRACKER["screen_capture_scale"]),
                    int(h * EYETRACKER["screen_capture_scale"]),
                ),
            )
            
            window_name = "Eyetracker"
            cv2.namedWindow(window_name, cv2.WINDOW_NORMAL)
            cv2.setWindowProperty(window_name, cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)
            cv2.imshow(window_name, screenshot)

            if EYETRACKER["write_to_disk"]:
                videoWriter.write(screenshot)

    if EYETRACKER["write_to_disk"]:
        videoWriter.release()

Starting face detector...
